In [1]:
# --- Librerías ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from scipy.stats import pointbiserialr
from sklearn.preprocessing import PowerTransformer

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.base import BaseEstimator
import os

# Configuración de gráficos
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (8, 5)

In [2]:
import logging
import pandas as pd
from scipy.stats import pointbiserialr
from sklearn.preprocessing import PowerTransformer

# Configuración del logger
logger = logging.getLogger('preprocessor_logger')
logger.setLevel(logging.INFO)

if not logger.handlers:
    handler = logging.StreamHandler()
    formatter = logging.Formatter(
        '%(asctime)s - %(name)s - %(levelname)s - [%(funcName)s] - %(message)s'
    )
    handler.setFormatter(formatter)
    logger.addHandler(handler)


class Preprocessor:
    def __init__(self, df: pd.DataFrame, target_col: str = 'target'):
        """
        Inicializa el preprocesador con el DataFrame original.
        
        Args:
            df: DataFrame a procesar
            target_col: Nombre de la columna target
        """
        self.target_col = target_col
        self.original_df = df.copy()
        self.processed_df = None
        self.cor_target = None
        self.product_cols = None
        
        # Atributos para exportar
        self.zones_ = None
        self.zone_mapper_df = None
        self.cols_to_drop_ = []
        self.power_params_ = None
        self.skewed_cols_ = []
        self.high_corr_df_products = None
        
        logger.info(f"Preprocessor inicializado - Shape original: {self.original_df.shape}")

    def _log_shape_change(self, previous_shape: tuple, operation: str):
        """Registra cambios en el shape del DataFrame."""
        current_shape = self.processed_df.shape
        logger.info(
            f"{operation} - Shape cambiado de {previous_shape} a {current_shape} "
            f"(Filas: {previous_shape[0]} → {current_shape[0]}, "
            f"Columnas: {previous_shape[1]} → {current_shape[1]})"
        )

    def group_sociodemographic_cols(self, sociodemographic_cols: list):
        """
        Agrupa columnas sociodemográficas en una sola columna 'zone'.
        
        Args:
            sociodemographic_cols: Lista de columnas sociodemográficas a agrupar
        """
        logger.info(f"Iniciando agrupación de columnas sociodemográficas: {sociodemographic_cols}")
        
        previous_shape = self.original_df.shape
        
        # Crear columna zone basada en agrupación
        self.original_df['zone'] = self.original_df.groupby(sociodemographic_cols).ngroup() + 1
        
        # Crear mapper de zonas
        expanded_cols = sociodemographic_cols + ['zone']
        self.zone_mapper_df = self.original_df[expanded_cols].drop_duplicates()
        
        self.zones_ = [str(col) for col in self.zone_mapper_df['zone'].unique()]
        
        # Eliminar columnas originales
        self.original_df.drop(columns=sociodemographic_cols, inplace=True)
        self.processed_df = self.original_df.copy()
        
        logger.info(f"Agrupación completada - {len(self.zone_mapper_df)} zonas únicas creadas")
        self._log_shape_change(previous_shape, "Agrupación sociodemográfica")

    def remove_duplicates(self):
        """Elimina duplicados exactos del DataFrame."""
        logger.info("Iniciando eliminación de duplicados exactos")
        
        previous_shape = self.processed_df.shape
        initial_rows = len(self.processed_df)
        
        self.processed_df = self.processed_df.drop_duplicates()
        final_rows = len(self.processed_df)
        removed_rows = initial_rows - final_rows
        
        logger.info(f"Duplicados eliminados: {removed_rows} filas removidas")
        self._log_shape_change(previous_shape, "Eliminación de duplicados exactos")

    def handle_complex_duplicates(self):
        """
        Maneja duplicados complejos donde las filas son idénticas excepto por el target.
        Conserva la moda del target en casos de múltiples duplicados.
        """
        logger.info("Iniciando manejo de duplicados complejos")
        
        df = self.processed_df.copy()
        previous_shape = df.shape
        initial_rows = len(df)
        
        # Agrupar filas duplicadas excepto por target
        columns = list(set(df.columns) - {self.target_col})
        grupos_duplicados = df.groupby(columns).groups
        
        indices_a_eliminar = []
        grupos_procesados = 0
        
        for fila, indices in grupos_duplicados.items():
            if len(indices) > 1:  # Solo grupos con duplicados
                grupos_procesados += 1
                grupo_actual = df.loc[indices]
                
                if len(indices) == 2:
                    # Eliminar todo el grupo (ambas filas)
                    indices_a_eliminar.extend(indices)
                else:
                    # Grupos con más de 2 filas: conservar solo la moda
                    moda_target = grupo_actual[self.target_col].mode()
                    
                    if len(moda_target) > 0:
                        moda = moda_target[0]
                        # Conservar solo las filas con target = moda
                        filas_a_eliminar = grupo_actual[grupo_actual[self.target_col] != moda].index
                        indices_a_eliminar.extend(filas_a_eliminar)
                    else:
                        # Si no hay moda clara, eliminar todo el grupo
                        indices_a_eliminar.extend(indices)
        
        # Crear nuevo DataFrame sin los duplicados problemáticos
        self.processed_df = df.drop(indices_a_eliminar)
        final_rows = len(self.processed_df)
        removed_rows = initial_rows - final_rows
        
        logger.info(
            f"Manejo de duplicados complejos completado - "
            f"{grupos_procesados} grupos procesados, {removed_rows} filas eliminadas"
        )
        self._log_shape_change(previous_shape, "Manejo de duplicados complejos")

    def get_correlations(self):
        """Calcula correlaciones punto-biserial entre features y target."""
        logger.info("Calculando correlaciones con el target")
        
        if self.target_col not in self.processed_df.columns:
            raise ValueError(f"Target column '{self.target_col}' no encontrada en el DataFrame")
        
        cor_target = {}
        features = [col for col in self.processed_df.columns if col != self.target_col]

        for col in features:
            corr, _ = pointbiserialr(self.processed_df[col], self.processed_df[self.target_col])
            cor_target[col] = corr

        self.cor_target = pd.Series(cor_target).sort_values(key=abs, ascending=False)
        
        logger.info(
            f"Correlaciones calculadas - "
            f"Rango: [{self.cor_target.min():.3f}, {self.cor_target.max():.3f}], "
            f"Top 3: {self.cor_target.head(3).to_dict()}"
        )

    def get_high_pair_correlations(self, product_cols: list):
        """
        Identifica pares de variables con alta correlación entre sí.
        
        Args:
            product_cols: Lista de columnas de productos a analizar
        """
        logger.info(f"Buscando correlaciones altas entre {len(product_cols)} columnas de productos")
        
        # Verificar que las columnas existen
        missing_cols = set(product_cols) - set(self.processed_df.columns)
        if missing_cols:
            raise ValueError(f"Columnas no encontradas: {missing_cols}")
        
        corr_abs_products = self.processed_df[product_cols].corr().abs()

        high_corr_pairs_products = []
        for i in range(len(corr_abs_products.columns)):
            for j in range(i+1, len(corr_abs_products.columns)):
                correlation = corr_abs_products.iloc[i, j]
                if correlation > 0.7:
                    high_corr_pairs_products.append({
                        'var1': corr_abs_products.columns[i],
                        'var2': corr_abs_products.columns[j], 
                        'correlation': correlation
                    })

        self.high_corr_df_products = pd.DataFrame(high_corr_pairs_products).sort_values(
            'correlation', ascending=False
        )
        
        # Filtrar correlaciones muy altas
        high_corr_count = len(self.high_corr_df_products[self.high_corr_df_products['correlation'] > 0.95])
        self.product_cols = product_cols
        
        logger.info(
            f"Análisis de correlación completado - "
            f"{len(high_corr_pairs_products)} pares con correlación > 0.7, "
            f"{high_corr_count} pares con correlación > 0.95"
        )

    def drop_high_correlated_cols(self):
        """Elimina columnas altamente correlacionadas, conservando las más relevantes."""
        logger.info("Iniciando eliminación de columnas altamente correlacionadas")
        
        if self.high_corr_df_products is None:
            raise ValueError("Debe ejecutar get_high_pair_correlations primero")
        
        previous_shape = self.processed_df.shape
        initial_cols = len(self.processed_df.columns)
        
        high_corr_filtered = self.high_corr_df_products[self.high_corr_df_products['correlation'] > 0.95]
        
        for _, row in high_corr_filtered.iterrows():
            var1 = row['var1']
            var2 = row['var2']

            # Comparar la correlación absoluta con el target
            corr_var1 = abs(self.cor_target[var1])
            corr_var2 = abs(self.cor_target[var2])

            # Quedarse con la columna más correlacionada, eliminar la otra
            if corr_var1 < corr_var2:
                col_to_drop = var1
                col_to_keep = var2
            else:
                col_to_drop = var2
                col_to_keep = var1
            
            if col_to_drop not in self.cols_to_drop_:
                self.cols_to_drop_.append(col_to_drop)
                logger.debug(f"Marcada para eliminar: {col_to_drop} (corr: {corr_var1:.3f}) "
                           f"vs {col_to_keep} (corr: {corr_var2:.3f})")

        self.cols_to_drop_ = list(set(self.cols_to_drop_))
        self.product_cols = list(set(product_cols)-set(self.cols_to_drop_))
        
        # Eliminar columnas
        columns_before_drop = set(self.processed_df.columns)
        self.processed_df = self.processed_df.drop(columns=self.cols_to_drop_)
        columns_after_drop = set(self.processed_df.columns)
        dropped_columns = columns_before_drop - columns_after_drop
        
        final_cols = len(self.processed_df.columns)
        
        logger.info(
            f"Eliminación de columnas correlacionadas completada - "
            f"{len(dropped_columns)} columnas eliminadas: {list(dropped_columns)}"
        )
        self._log_shape_change(previous_shape, "Eliminación de columnas correlacionadas")

    def correct_skewness(self):
        """
        Corrige asimetría en las columnas usando transformación Yeo-Johnson.
        
        Args:
            product_cols: Lista de columnas de productos a transformar
        """
        logger.info("Iniciando corrección de asimetría")
        
        # Calcular asimetría inicial
        skewness_before = self.processed_df[self.product_cols].skew()
        self.skewed_cols_ = skewness_before[abs(skewness_before) > 0.5].index.tolist()
        
        if not self.skewed_cols_:
            logger.info("No se encontraron columnas con asimetría significativa (> 0.5)")
            return
            
        logger.info(f"{len(self.skewed_cols_)} columnas con asimetría > 0.5: {self.skewed_cols_}")
        
        # Aplicar transformación Yeo-Johnson
        pt = PowerTransformer(method='yeo-johnson')
        self.processed_df[self.skewed_cols_] = pt.fit_transform(self.processed_df[self.skewed_cols_])
        
        # Calcular asimetría después de la transformación
        skewness_after = self.processed_df[self.skewed_cols_].skew()
        
        # Guardar parámetros
        if hasattr(pt, "lambdas_"):
            self.power_params_ = dict(zip(self.skewed_cols_, pt.lambdas_))
        else:
            self.power_params_ = {}
        
        logger.info(
            f"Corrección de asimetría completada - "
            f"Asimetría promedio: {skewness_before.mean():.3f} → {skewness_after.mean():.3f}"
        )

    def apply_one_hot(self):
        """Aplica one-hot encoding a la columna 'zone'."""
        logger.info("Aplicando one-hot encoding a la columna 'zone'")
        
        if 'zone' not in self.processed_df.columns:
            raise ValueError("Columna 'zone' no encontrada para one-hot encoding")
        
        previous_shape = self.processed_df.shape
        
        # Convertir a entero y aplicar one-hot
        self.processed_df['zone'] = self.processed_df['zone'].astype(int)
        zone_dummies = pd.get_dummies(self.processed_df['zone'], prefix='zone')
        
        # Concatenar y eliminar columna original
        self.processed_df = pd.concat([
            self.processed_df.drop('zone', axis=1), 
            zone_dummies
        ], axis=1)
        
        logger.info(f"One-hot encoding completado - {len(zone_dummies.columns)} columnas zone creadas")
        self._log_shape_change(previous_shape, "One-hot encoding")

    def apply_preprocess(self, sociodemographic_cols: list, product_cols: list) -> pd.DataFrame:
        """
        Ejecuta el pipeline completo de preprocesamiento.
        
        Args:
            sociodemographic_cols: Columnas sociodemográficas a agrupar
            product_cols: Columnas de productos para análisis de correlación
            
        Returns:
            DataFrame procesado
        """
        logger.info("=== INICIANDO PIPELINE COMPLETO DE PREPROCESAMIENTO ===")
        logger.info(f"Columnas sociodemográficas: {sociodemographic_cols}")
        logger.info(f"Columnas de productos: {product_cols}")
        
        # Pipeline de procesamiento
        self.group_sociodemographic_cols(sociodemographic_cols)
        self.remove_duplicates()
        self.handle_complex_duplicates()
        self.get_correlations()
        self.get_high_pair_correlations(product_cols)
        self.drop_high_correlated_cols()
        self.correct_skewness()
        self.apply_one_hot()
        
        logger.info("=== PIPELINE COMPLETADO EXITOSAMENTE ===")
        logger.info(f"Shape final del DataFrame: {self.processed_df.shape}")
        logger.info(f"Columnas eliminadas: {len(self.cols_to_drop_)}")
        logger.info(f"Columnas transformadas: {len(self.skewed_cols_)}")
        
        return self.processed_df

    def get_preprocessing_summary(self) -> dict:
        """Retorna un resumen del proceso de preprocesamiento."""
        return {
            'original_shape': self.original_df.shape,
            'processed_shape': self.processed_df.shape if self.processed_df is not None else None,
            'zones_created': len(self.zone_mapper_df) if self.zone_mapper_df is not None else 0,
            'columns_dropped': len(self.cols_to_drop_),
            'columns_skewness_corrected': len(self.skewed_cols_),
            'high_correlation_pairs': len(self.high_corr_df_products) if self.high_corr_df_products is not None else 0
        }

In [3]:
os.chdir("../..")  # subir al nivel raíz del proyecto
!pwd  # verificar ruta actual



/Users/luisfernandocaporalmontesdeoca/Documents/MNA-V Tec de Monterrey/MLOps/MNA_MLOps_Equipo12_backup


In [4]:
!dvc pull data/raw/insurance_company_original.csv.dvc


Fetching
!
  0% Checking cache in '/Users/luisfernandocaporalmontesdeoca/Documents/MNA-V Te
Fetching                                                                        
Building workspace index                              |3.00 [00:00,  431entry/s]
Comparing indexes                                    |4.00 [00:00, 1.38kentry/s]
Applying changes                                      |0.00 [00:00,     ?file/s]
Everything is up to date.


In [5]:
os.chdir("notebooks")


file_name = '../data/raw/insurance_company_original.csv'
sociodemographic_cols = [f"SD_{i}" for i in range(1, 44)]
product_cols = [f"PD_{i-44}" for i in range(44, 86)]
cols = sociodemographic_cols + product_cols + ["target"]
df = pd.read_csv(file_name, header=None, names=cols)
df = df.iloc[1:].reset_index(drop=True)


In [6]:
preprocessor = Preprocessor(df)
processed = preprocessor.apply_preprocess(sociodemographic_cols, product_cols)

2025-11-11 16:54:14,797 - preprocessor_logger - INFO - [__init__] - Preprocessor inicializado - Shape original: (5821, 86)
2025-11-11 16:54:14,798 - preprocessor_logger - INFO - [apply_preprocess] - === INICIANDO PIPELINE COMPLETO DE PREPROCESAMIENTO ===
2025-11-11 16:54:14,799 - preprocessor_logger - INFO - [apply_preprocess] - Columnas sociodemográficas: ['SD_1', 'SD_2', 'SD_3', 'SD_4', 'SD_5', 'SD_6', 'SD_7', 'SD_8', 'SD_9', 'SD_10', 'SD_11', 'SD_12', 'SD_13', 'SD_14', 'SD_15', 'SD_16', 'SD_17', 'SD_18', 'SD_19', 'SD_20', 'SD_21', 'SD_22', 'SD_23', 'SD_24', 'SD_25', 'SD_26', 'SD_27', 'SD_28', 'SD_29', 'SD_30', 'SD_31', 'SD_32', 'SD_33', 'SD_34', 'SD_35', 'SD_36', 'SD_37', 'SD_38', 'SD_39', 'SD_40', 'SD_41', 'SD_42', 'SD_43']
2025-11-11 16:54:14,799 - preprocessor_logger - INFO - [apply_preprocess] - Columnas de productos: ['PD_0', 'PD_1', 'PD_2', 'PD_3', 'PD_4', 'PD_5', 'PD_6', 'PD_7', 'PD_8', 'PD_9', 'PD_10', 'PD_11', 'PD_12', 'PD_13', 'PD_14', 'PD_15', 'PD_16', 'PD_17', 'PD_18', '

In [9]:
import mlflow
import mlflow.sklearn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [11]:
target = "target"
X = processed.drop(columns=[target])
y = processed[target]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Para la regresión logistica
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [12]:

def train_and_log_model(model, model_name, X_train, X_test, y_train, y_test, params=None):
    with mlflow.start_run(run_name=model_name):
        if params:
            mlflow.log_params(params)
        
        # Entrenamiento
        model.fit(X_train, y_train)
        
        # Predicciones
        y_pred = model.predict(X_test)
        y_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None
        
        # Métricas
        metrics = {
            "accuracy": accuracy_score(y_test, y_pred),
            "precision": precision_score(y_test, y_pred, zero_division=0),
            "recall": recall_score(y_test, y_pred, zero_division=0),
            "f1_score": f1_score(y_test, y_pred, zero_division=0),
        }
        if y_proba is not None:
            metrics["roc_auc"] = roc_auc_score(y_test, y_proba)

        mlflow.log_metrics(metrics)
        
        # Log del modelo
        mlflow.sklearn.log_model(model, model_name)
        
        print(f"✅ {model_name} registrado en MLflow con métricas:")
        for k, v in metrics.items():
            print(f"   {k}: {v:.4f}")

# ============================================
# Entrenamiento de modelos
# ============================================

# --- Regresión Logística ---
log_reg_params = {"solver": "liblinear", "random_state": 42}
log_reg = LogisticRegression(**log_reg_params)
train_and_log_model(log_reg, "LogisticRegression", X_train_scaled, X_test_scaled, y_train, y_test, log_reg_params)

# --- Random Forest ---
rf_params = {"n_estimators": 200, "max_depth": 10, "random_state": 42}
rf = RandomForestClassifier(**rf_params)
train_and_log_model(rf, "RandomForest", X_train, X_test, y_train, y_test, rf_params)

# --- XGBoost ---
xgb_params = {
    "n_estimators": 300,
    "max_depth": 5,
    "learning_rate": 0.05,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "random_state": 42,
    "use_label_encoder": False,
    "eval_metric": "logloss",
}
xgb = XGBClassifier(**xgb_params)
train_and_log_model(xgb, "XGBoost", X_train, X_test, y_train, y_test, xgb_params)


/Users/luisfernandocaporalmontesdeoca/miniconda3/envs/MNA_MLOps_Equipo12_backup/lib/python3.11/site-packages/mlflow/tracking/_tracking_service/utils.py:140: FutureWarning: Filesystem tracking backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri, store_uri)
2025/11/11 17:11:27 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/11 17:11:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ LogisticRegression registrado en MLflow con métricas:
   accuracy: 0.9288
   precision: 0.2000
   recall: 0.0862
   f1_score: 0.1205
   roc_auc: 0.6732


2025/11/11 17:11:30 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/11 17:11:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ RandomForest registrado en MLflow con métricas:
   accuracy: 0.9434
   precision: 0.0000
   recall: 0.0000
   f1_score: 0.0000
   roc_auc: 0.7500


/Users/luisfernandocaporalmontesdeoca/miniconda3/envs/MNA_MLOps_Equipo12_backup/lib/python3.11/site-packages/xgboost/training.py:199: UserWarning: [17:11:32] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
2025/11/11 17:11:34 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/11 17:11:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ XGBoost registrado en MLflow con métricas:
   accuracy: 0.9434
   precision: 0.5000
   recall: 0.0172
   f1_score: 0.0333
   roc_auc: 0.7498
